# Estudo da composição das previsões do rating final

O objetivo é realizar previsões para toda base gerando um rating unindo o contribuinte e posteriormente o igr gerado na regressão

In [1]:
import os
import dotenv
import boto3
from io import BytesIO
import pandas as pd
from datetime import date
import zipfile
from sklearn.preprocessing import StandardScaler, RobustScaler
import pickle

import warnings

warnings.filterwarnings("ignore")

print("Processo iniciado")

# Define diretorios
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

Processo iniciado


True

In [2]:
s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
)

In [4]:
print("Carregando dados para previsões")

# Prepara os dados para realizar as previsões

zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)


def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df


base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')
# base_parcelas = ler_bases_exportadas('parcelas.csv')

Carregando dados para previsões


In [5]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento', 'da_aberto'],
      dtype='object')

In [6]:
print("Inicia transformação das variáveis sobre a dívida")
print("Processamento das variáveis sobre a dívida")

# Gera as variáveis de tempo
base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

Inicia transformação das variáveis sobre a dívida
Processamento das variáveis sobre a dívida


In [7]:
# Seleciona dados sobre a divida
dados_divida = base_conjunta[
    ['cda', 'id_contribuinte', 'cpf_cnpj_existe',
     'da_aberto', 'atividade_principal', 'valor_tot','vlr_pago', 'vlr_tributo', 'vlr_taxa', 'arrecadacao_divida',
     'ajuizamento_divida', 'ajuizamento', 'protesto', 'competencia_divida', 'refis',
     'tipo_divida', 'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao']]
dados_divida.dropna(subset=['id_contribuinte'], inplace=True)
dados_divida['id_contribuinte'] = dados_divida['id_contribuinte'].astype(str)  # persistindo tipo de dados

In [8]:
# Calcula a idade da dívida ativa
dados_divida['ano_atual'] = date.today().year
dados_divida['anos_idade_da'] = dados_divida['ano_atual'] - dados_divida['ano_inscricao_da']
dados_divida = dados_divida.drop(columns=['ano_atual'])

In [9]:
# Renomeia colunas para nome mais adequados e filtra dataframe
colunas_nome = {
    'valor_tot': 'valor_total_da',
}
df_divida_ativa = dados_divida.rename(columns=colunas_nome)

In [ ]:
# Obtem os dados de parcelamento das dívidas ativas

# df_parcelas = base_parcelas[['cda', 'id_pessoa', 'tipo_divida', 'quantidade_reparcelamentos']]
# df_parcelas.drop_duplicates(subset='cda', inplace=True)
# df_parcelas.dropna(subset='cda', inplace=True)

# df_divida_ativa = pd.merge(
#         df_divida_ativa,    
#         df_parcelas,    
#         how='left',    
#         left_on=['cda', 'tipo_divida', 'id_contribuinte'],    
#         right_on=['cda', 'tipo_divida', 'id_pessoa'],    
#         suffixes=('', '_PARC'))

In [10]:
print("Processamento das variáveis sobre o contribuinte")

# Cria conexão com o banco e prepara os dados

def read_s3_files(bucket_name, folder_name, file_name):
    file_key_aws = folder_name + file_name
    obj = s3.Bucket(bucket_name).Object(file_key_aws).get()
    df = pd.read_csv(obj['Body'], sep=';')
    return df

dados_contribuinte = read_s3_files(bucket_name=os.getenv("S3_BUCKET_NAME"), folder_name=os.getenv("S3_FOLDER_NAME"),
                                   file_name='feature_store_contribuinte_2.csv')


Processamento das variáveis sobre o contribuinte


In [14]:
df_divida_ativa.columns

Index(['cda', 'id_contribuinte', 'cpf_cnpj_existe', 'da_aberto',
       'atividade_principal', 'valor_total_da', 'vlr_pago', 'vlr_tributo',
       'vlr_taxa', 'arrecadacao_divida', 'ajuizamento_divida', 'ajuizamento',
       'protesto', 'competencia_divida', 'refis', 'tipo_divida',
       'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao',
       'anos_idade_da'],
      dtype='object')

In [15]:
dados_contribuinte.columns

Index(['id_pessoa', 'situacao', 'cpf_cnpj_existe', 'edificacao',
       'qtd_notas_2anos', 'situacao_ativa', 'status_situacao', 'deb_totais',
       'deb_pagos', 'valor_tot', 'vlr_pago', 'frequencia_da_pessoa',
       'total_debitos_pessoa', 'debitos_pagos_pessoa', 'valor_total_pessoa',
       'valor_pago_pessoa', 'historico_pagamento_em_qtd',
       'historico_pagamento_em_valor', 'class_contribuinte',
       'class_contribuinte_nome', 'class_contribuinte_peso'],
      dtype='object')

In [20]:
    contribuinte_aux =  dados_contribuinte[['id_pessoa', 'edificacao',
                                            'qtd_notas_2anos', 'situacao_ativa', 'status_situacao',
                                             'frequencia_da_pessoa', 'historico_pagamento_em_qtd',
                                             'historico_pagamento_em_valor', 'class_contribuinte',
                                            'class_contribuinte_nome', 'class_contribuinte_peso']]

In [ ]:
contribuinte_aux

In [24]:
# Realiza o merge dos dados da dívida com os dados do contribuinte

df = pd.merge(
    left=df_divida_ativa,
    right=contribuinte_aux,
    left_on='id_contribuinte',
    right_on='id_pessoa',
    how='left')

In [25]:
df.columns

Index(['cda', 'id_contribuinte', 'cpf_cnpj_existe', 'da_aberto',
       'atividade_principal', 'valor_total_da', 'vlr_pago', 'vlr_tributo',
       'vlr_taxa', 'arrecadacao_divida', 'ajuizamento_divida', 'ajuizamento',
       'protesto', 'competencia_divida', 'refis', 'tipo_divida',
       'ano_inscricao_da', 'quantidade_reparcelamento', 'situacao',
       'anos_idade_da', 'id_pessoa', 'edificacao', 'qtd_notas_2anos',
       'situacao_ativa', 'status_situacao', 'frequencia_da_pessoa',
       'historico_pagamento_em_qtd', 'historico_pagamento_em_valor',
       'class_contribuinte', 'class_contribuinte_nome',
       'class_contribuinte_peso'],
      dtype='object')

In [26]:
# Ajustando pesos dos contribuintes, foi dado ao primeira dívida o mesmo peso do bom pagador, isso procede?
df.loc[df['frequencia_da_pessoa'] <= 1, 'class_contribuinte_nome'] == 'PRIMEIRA DIVIDA'
df.loc[df['class_contribuinte_nome'] == 'PRIMEIRA DIVIDA', 'class_contribuinte_peso'] == 2.54487

df['class_contribuinte_nome'] = df['class_contribuinte_nome'].fillna('PIOR PAGADOR')
df.loc[df['class_contribuinte_nome'] == 'PIOR PAGADOR', 'class_contribuinte_peso'] == -0.98031
df.fillna(0, inplace=True)

In [27]:
print("Preparação do pipeline de previsões")
# Filtra os dados que precisamos para previsão
df_feature_store = df[['valor_total_da', 'anos_idade_da', #'quantidade_reparcelamentos',
                       'frequencia_da_pessoa', 'historico_pagamento_em_qtd', 'status_situacao',
                       'historico_pagamento_em_valor', 'class_contribuinte_peso']]

Preparação do pipeline de previsões


In [28]:
# Normaliza os dados para previsão utilizando o StandardScaler
normalizador = RobustScaler()
normalizador.fit(df_feature_store)
dados_normalizados = normalizador.fit_transform(df_feature_store)

colunas = list(normalizador.get_feature_names_out())
df_normalizado = pd.DataFrame(dados_normalizados, columns=colunas)

In [29]:
print("Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)")


def abre_modelo(nome_modelo, path_modelo, zip_name=None):
    if zip_name:
        zip_file = os.path.join(path_modelo, zip_name)
        z = zipfile.ZipFile(zip_file)
        z.extract(nome_modelo)
    else:
        nome_modelo = os.path.join(path_modelo, nome_modelo)

    modelo = pickle.load(open(nome_modelo, 'rb'))
    return modelo


model_predict_igr = abre_modelo("modeloDA-igr-divida-v2.pkl", modelsPath)

Carrega o modelo e realiza a previsão do Índice Geral de Recuperação (IGR)


In [30]:
# Realizando previsões
previsoes = model_predict_igr.predict(df_normalizado)
df['igr'] = previsoes
df.loc[df['status_situacao'] == 0, 'igr'] = 0
df.loc[df['anos_idade_da'] >= 15, 'igr'] = 0

print("Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte")

def make_rating_divida(dataframe):
    dataframe.loc[dataframe['igr'] == 0, 'rating_divida'] = 'BAIXISSIMA'

    # Melhor Pagador
    dataframe.loc[(dataframe['rating_divida'].isnull()) & (dataframe['class_contribuinte'] == 2), 'rating_divida'] = 'ALTISSIMA'

    # Pior Pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 0) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.1), 'rating_divida'] = 'ALTA'
    # dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Pagador intermediario
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 1) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'

    # Bom pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] >= 0.5), 'rating_divida'] = 'ALTISSIMA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.5) & (dataframe['igr'] >= 0.05), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 3) & (dataframe['igr'] < 0.05) & (dataframe['igr'] != 0), 'rating_divida'] = 'MEDIA'

    # Melhor Pagador
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] >= 0.3), 'rating_divida'] = 'ALTA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.3) & dataframe['igr'] >= 0.1, 'rating_divida'] = 'MEDIA'
    dataframe.loc[(dataframe['class_contribuinte'] == 4) & (dataframe['igr'] < 0.1) & (dataframe['igr'] != 0), 'rating_divida'] = 'BAIXA'
    return dataframe


df = make_rating_divida(df)

Início do processo de classificação do rating da dívida parametrizando junto a classificação do contribuinte


In [31]:
print("Inicia a conexão com S3 para inscrição dos dados com as previsões")

# Cria conexão ao s3 e preenche a tabela com os dados

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())


up_s3_files(dataframe=df,
            bucket_name=os.getenv("S3_BUCKET_NAME"),
            folder_name=os.getenv("S3_FOLDER_NAME"),
            file_name='baseline_beta.csv')

print("Upload de dados efetuados no s3")
print("Processo finalizado")
print("Arquivo disponível para download e análise")

Inicia a conexão com S3 para inscrição dos dados com as previsões
Upload de dados efetuados no s3
Processo finalizado
Arquivo disponível para download e análise
